## 목차
### 1. Library Import
### 2. 학습데이터 생성
### 3. Light-gbm 모델 훈련
### 4. 교차 검증 점수 확인
### 5. 제출 파일 생성

# 1. Library Import

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')


import os, sys 
from google.colab import drive 
drive.mount('/content/mnt') 
nb_path = '/content/notebooks' 
os.symlink('/content/mnt/', nb_path) 
sys.path.insert(0, nb_path)

Mounted at /content/mnt


In [3]:
# 자료형 맞춰주는 함수 정의
def make_datetime(x):
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #min  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)
    # return dt.datetime(month, day)

def string2num(x):
  # , () 같은 구분자 없애서 데이터 정제
  x = re.sub(r"[^0-9]+", '', str(x))
  if x =='':
    return 0
  else:
    return int(x)

In [4]:
PATH = '/content/mnt/MyDrive/data/'

Train Error 가져오기

In [5]:
train_err = pd.read_csv(PATH + '/train_err_data.csv', thousands=',')

FileNotFoundError: ignored

## Quality Data 분석해보기



In [ ]:
train_qual = pd.read_csv(PATH + '/train_quality_data.csv', thousands=',')
train_qual.fillna(0, inplace=True) # nan값이 있으면 int64로 변환이 안돼서 nan은 0으로 변경, 'inplace=True'가 있어야 값이 바뀌어서 들어감

train_qual['quality_0']=train_qual['quality_0'].astype('int64')
train_qual['quality_2']=train_qual['quality_2'].astype('int64')
train_qual['quality_5']=train_qual['quality_5'].astype('int64')


In [ ]:
# 성진이 코드
# user별 퀄리티의 min max

# train_qual = pd.read_csv(PATH+'train_quality_data.csv',thousands=',')
train_qual['datetime'] = train_qual['time'].apply(make_datetime)

id_qual=train_qual[['user_id','datetime','fwver','quality_0','quality_1','quality_2','quality_3','quality_4','quality_5','quality_6','quality_7','quality_8','quality_9','quality_10','quality_11','quality_12']].values
qualili=np.zeros((15000,14))
curuser=10000
cnt=0
#qualist=([10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500])*15000
qualist=np.zeros((15000,26))
for i in qualist:
    for j in range(13):
        i[j]=100000000
    for j in range(13,26):
        i[j]=-500

min=1000000000
max=-500
for j in tqdm(id_qual):
    if curuser==j[0]:
        for i in range(3,16):
          if( qualist[curuser-10000][i-3]>j[i]):
              qualist[curuser-10000][i-3]=j[i]
          elif (qualist[curuser-10000][i-3+13]<j[i]):
              qualist[curuser-10000][i-3+13]=j[i]
    else:
        curuser=j[0]
        for i in range(3,16):
          if( qualist[curuser-10000][i-3]>j[i]):
              qualist[curuser-10000][i-3]=j[i]
          elif (qualist[curuser-10000][i-3+13]<j[i]):
              qualist[curuser-10000][i-3+13]=j[i]

#for user,dat,fw,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12 in tqdm(id_qual):
print(curuser)
newqualist=np.zeros((15000,13))
for i in range(15000):
    for j in range(13):
        if (qualist[i][j+13]-qualist[i][j] <-5000): #초기값
            continue
        else:
            newqualist[i][j]=qualist[i][j+13]-qualist[i][j]

   

In [ ]:
print(newqualist)
print(newqualist[2])
newqualist.shape

In [ ]:
# quality 3과 4는 0 값만 있기 때문에 뺀다.
del train_qual['quality_3']
del train_qual['quality_4']

train_qual.dtypes

In [ ]:
train_quality.info()

In [ ]:
train_qual['fwver'] = train_qual['fwver'].str.replace(pat=r"^03.[0-9]+.[0-9]+", repl=r'03', regex=True)
train_qual['fwver'] = train_qual['fwver'].str.replace(pat=r"^04.[0-9]+.[0-9]+", repl=r'04', regex=True)
train_qual['fwver'] = train_qual['fwver'].str.replace(pat=r"^05.[0-9]+.[0-9]+", repl=r'05', regex=True)

train_qual['fwver'].value_counts()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

train_qual[:36]

In [ ]:

user_qual = np.sort(train_qual['user_id'].unique())
user_prob = np.sort(train_prob['user_id'].unique())

print("qualty ", user_qual)
print("problem ", user_prob)

# qualty에는 있지만 problem에는 없는 사람들 : 이상이 있었는데 신고할 정도는 아니었따. 5114명
qual_prob = list(set(user_qual) - set(user_prob))
print("qualty - problem ", qual_prob)
print(len(qual_prob))

# 프로블럼에는 있지만 퀄리티엔 없는 : 이상이 감지되진 않았는데 신고를 한거              1833명
prob_qual = list(set(user_prob) - set(user_qual))
print("problem - qualty ", prob_qual)
print(len(prob_qual))



In [ ]:
train_qual.describe()

In [ ]:
# 모든 컬럼 유니크 값 찍어보기 --> 너무 많고 복잡해
col_list = train_qual.columns
print(col_list)
for col in col_list:
  print(col)
  print(train_qual[col].unique())
  

In [ ]:
# print(np.sort(train_qual['quality_0'].unique()))

# newqualist['quality_12'].value_counts().sort_index().plot(color='b', lw=2, label='qual_0')
# print(newqualist)

plt.rcParams['figure.figsize'] = [12, 8]
plt.figure(1)
plt.plot(newqualist[10013:10023])
plt.figure(2)
plt.plot(newqualist[11222:11231])

plt.figure(3)
plt.plot(newqualist[332])

plt.figure(4)
plt.plot(newqualist[1341])

for i in [10023, 11231, 332, 1341]:
  print(newqualist[i])

plt.legend()    # 자동 범례
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.figure(1)
plt.plot(newqualist[0:30][12])

plt.figure(2)
# plt.plot(newqualist[0:30][10])
plt.figure(3)
# plt.plot(newqualist[0:30][11])

print(newqualist[0:10])
# plt.figure(2)
# plt.plot(newqualist[10:18])

# 0, 2 , 4, 5 ,6, 10, 12

plt.legend()    # 자동 범례
plt.show()

In [ ]:
# print(newqualist[:20][2])
# print(newqualist[:20][4])

outlier = []
normal = []
f = True
cnt = 0
for i in range(0,15000):
  for q in [0,2,4,5,6,10,12]:
    if (newqualist[i][q] > 100):
      cnt += 1
  
  # 중간에 안끝나고 5보다 작은 값으로 계속 간 유저가 있으면 
  # 그 유저는 안 중요한 유저로 (신고를 안할만한 유저로)
  # print(cnt)
  if (cnt <3):
    normal.append(i)
  else:
    outlier.append(i)

print(outlier)
print(normal)

In [ ]:
display(train_qual[(train_qual['user_id']==user)][{'user_id', 'time', 'quality_2'}].value_counts())


# 2. 학습 데이터 생성

In [ ]:
train_user_id_max = 24999   # 유저 아이디 끝 번호
train_user_id_min = 10000   # 유저 아이디 시작 번호
train_user_number = 15000   # 유저 수

## 2.1 train_err

* train_err_data.csv 파일의 errtype column의 value_counts를 카운트하여 학습에 사용.  
* 주어진 기간동안 user_id별로 peroblem가 한 번이라도 발생했는 지를 확인.  

In [ ]:
print(np.sort(train_err.errtype.unique()))
# errtype이 1부터 42까지 29를 제외한 41개가 존재.

In [ ]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

In [ ]:
#참고) 일별 합산 데이터 생성 코드      
'''
train_err             = pd.read_csv(PATH+'train_err_data.csv')
train_err['datetime'] = train_err['time'].apply(make_datetime)

min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

id_error = train_err[['user_id','errtype','days']].values
error = np.zeros((train_user_number,42, 33))
for idx, err, days in tqdm(id_error):
    error[idx - train_user_id_min,err - 1, days] += 1
error.shape
'''

## 2.2 problem

In [ ]:
train_prob = pd.read_csv(PATH+'/train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

### 내코드

In [ ]:
train_qual.head()

In [ ]:
# 성진이 코드
# user별 퀄리티의 min max

# train_qual = pd.read_csv(PATH+'train_quality_data.csv',thousands=',')
train_qual['datetime'] = train_qual['time'].apply(make_datetime)

id_qual=train_qual[['user_id','datetime','fwver','quality_0','quality_1','quality_2','quality_3','quality_4','quality_5','quality_6','quality_7','quality_8','quality_9','quality_10','quality_11','quality_12']].values
qualili=np.zeros((15000,14))
curuser=10000
cnt=0
#qualist=([10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500])*15000
qualist=np.zeros((15000,26))
for i in qualist:
    for j in range(13):
        i[j]=100000000
    for j in range(13,26):
        i[j]=-500

min=1000000000
max=-500
for j in tqdm(id_qual):
    if curuser==j[0]:
        for i in range(3,16):
          if( qualist[curuser-10000][i-3]>j[i]):
              qualist[curuser-10000][i-3]=j[i]
          elif (qualist[curuser-10000][i-3+13]<j[i]):
              qualist[curuser-10000][i-3+13]=j[i]
    else:
        curuser=j[0]
        for i in range(3,16):
          if( qualist[curuser-10000][i-3]>j[i]):
              qualist[curuser-10000][i-3]=j[i]
          elif (qualist[curuser-10000][i-3+13]<j[i]):
              qualist[curuser-10000][i-3+13]=j[i]

#for user,dat,fw,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12 in tqdm(id_qual):
print(curuser)
newqualist=np.zeros((15000,13))
for i in range(15000):
    for j in range(13):
        if (qualist[i][j+13]-qualist[i][j] <-5000): #초기값
            continue
        else:
            newqualist[i][j]=qualist[i][j+13]-qualist[i][j]

   

In [ ]:
diff_qual = pd.DataFrame(newqualist)

diff_qual.rename(columns={5:"quality_5"}, inplace = True)
diff_qual.rename(columns={10:"quality_10"}, inplace = True)


diff_qual['user_id'] = train_user_id_min + diff_qual.index
diff_qual.head()

# diff_qual.info()

In [ ]:
display(diff_qual, 10)

In [ ]:
prob_user_list = train_prob['user_id'].unique()

id_error = train_err[['user_id','errtype', 'errcode']].values

# print(id_error)
# active, timeout, fail to establish
error = np.zeros((train_user_number,42 + 3 + 2))

for person_idx, err, errcode in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1

    ######### 내 코드 #########
    if (err == 23):
      if (errcode == 'connection timeout'): error[person_idx - train_user_id_min, 42] += 1
      if (errcode == 'active'): error[person_idx - train_user_id_min, 43] += 1
      if (errcode == 'connection fail to establish'): error[person_idx - train_user_id_min, 44] += 1


In [ ]:
# 5 6 9 10
# quality 추가 5번 10번
id_qual = diff_qual[['user_id', 'quality_5', 'quality_10']].values

# id_qual.info()
id_qual = id_qual.astype(np.int64)

# print(id_qual)

for person_idx, q5, q10 in tqdm(id_qual):
  # print(person_idx, q5, q10)
  # print(type(person_idx), type(q5), type(q10))
  if (q5 > 10):
    error[int(person_idx - train_user_id_min), 45] += 1
  if (q10 > 10):
    error[int(person_idx - train_user_id_min), 46] += 1

In [ ]:
error.shape

In [ ]:
print(error)

In [ ]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = error
train_y = problem
del error, problem
print(train_x.shape)
print(train_y.shape)

In [ ]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params_old =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }

params = {'num_leaves':100,
       'min_data_in_leaf':30,
       'objective':'binary',
       'max_depth':6 ,
       'learning_rate':0.02 ,
       'boosting':"gbdt",
       "feature_fraction":0.7522,
       "bagging_freq":1,
       "bagging_fraction":0.7083,
       "bagging_seed":11,
       "metric":"rmse",
       "random_state":133
       }

#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 45
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

# 4. 교차검증 점수 확인

In [ ]:
print(np.mean(auc_scores))

# 5. 제출 파일 생성

In [ ]:
test_err  = pd.read_csv(PATH+'/test_err_data.csv')
display(test_err.head())

In [ ]:
# 성진이 코드

test_qual = pd.read_csv(PATH+'test_quality_data.csv',thousands=',')
test_qual['datetime'] = test_qual['time'].apply(make_datetime)

id_qual=test_qual[['user_id','datetime','fwver','quality_0','quality_1','quality_2','quality_3','quality_4','quality_5','quality_6','quality_7','quality_8','quality_9','quality_10','quality_11','quality_12']].values


In [ ]:
# test_qual.info()
# test_qual.head()
# display(test_qual)

print(test_qual['user_id'].min(), test_qual['user_id'].max())

In [ ]:
qualili=np.zeros((15000,14))
curuser=10000
cnt=0
#qualist=([10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500])*15000
qualist=np.zeros((15000,26))
for i in qualist:
    for j in range(13):
        i[j]=100000000
    for j in range(13,26):
        i[j]=-500
minmin = 30000
min=1000000000
max=-500
for j in tqdm(id_qual):
    if curuser==j[0]:
        for i in range(3,16):
          if( qualist[curuser-minmin][i-3]>j[i]):
              qualist[curuser-minmin][i-3]=j[i]
          elif (qualist[curuser-minmin][i-3+13]<j[i]):
              qualist[curuser-minmin][i-3+13]=j[i]
    else:
        curuser=j[0]
        for i in range(3,16):
          if( qualist[curuser-minmin][i-3]>j[i]):
              qualist[curuser-minmin][i-3]=j[i]
          elif (qualist[curuser-minmin][i-3+13]<j[i]):
              qualist[curuser-minmin][i-3+13]=j[i]

#for user,dat,fw,q0,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12 in tqdm(id_qual):
print(curuser)
newqualist=np.zeros((15000,13))
for i in range(15000):
    for j in range(13):
        if (qualist[i][j+13]-qualist[i][j] <-5000): #초기값
            continue
        else:
            newqualist[i][j]=qualist[i][j+13]-qualist[i][j]

   

In [ ]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [ ]:
diff_qual = pd.DataFrame(newqualist)

diff_qual.rename(columns={5:"quality_5"}, inplace = True)
diff_qual.rename(columns={10:"quality_10"}, inplace = True)


diff_qual['user_id'] = test_user_id_min + diff_qual.index
diff_qual.head()

In [ ]:
id_error = test_err[['user_id','errtype', 'errcode']].values
test_x = np.zeros((test_user_number,42 + 3 + 2))

for person_idx, err, errcode in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1

    ######### 내 코드 #########
    if (err == 23):
      if (errcode == 'connection timeout'): test_x[person_idx - test_user_id_min, 42] += 1
      if (errcode == 'active'): test_x[person_idx - test_user_id_min, 43] += 1
      if (errcode == 'connection fail to establish'): test_x[person_idx - test_user_id_min, 44] += 1


# 5 6 9 10
# quality 추가 5번 10번
id_qual = diff_qual[['user_id', 'quality_5', 'quality_10']].values

# id_qual.info()
id_qual = id_qual.astype(np.int64)

# print(id_qual)

for person_idx, q5, q10 in tqdm(id_qual):
  # print(person_idx, q5, q10)
  # print(type(person_idx), type(q5), type(q10))
  if (q5 > 10):
    test_x[int(person_idx - test_user_id_min), 45] += 1
  if (q10 > 10):
    test_x[int(person_idx - test_user_id_min), 46] += 1

    

In [ ]:
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

In [ ]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]

feature_importance=pd.DataFrame([model.feature_name(),model.feature_importance()]).T
feature_importance.columns=["features","varimp"]
feature_importance=feature_importance.sort_values(["varimp"],ascending=False)
feature_importance.head()
feature_importance.to_csv(PATH+"/importance.csv",index=False)

lgb.plot_importance(model)
plt.figure(figsize=(15,20))
plt.show()

In [ ]:
pred_ensemble

In [ ]:
sample_submssion = pd.read_csv(PATH+'/sample_submission.csv')

In [ ]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [ ]:
sample_submssion.to_csv(PATH + "/yunji_10.csv", index = False)

In [ ]:
sample_submssion.head()